<a href="https://colab.research.google.com/github/gab-nlp/pruebas-yahoo/blob/main/Gensim_Yahoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Monitorizar

In [59]:
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

## Comprobar uso de GPU

In [1]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


## Importar librerías generales y abrir fichero csv

In [2]:
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
nRowsRead = 1000 # specify 'None' if want to read whole file
test_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/Yahoo_Answers/train.csv', delimiter=',', header=None,#nrows = nRowsRead,
                  names=['categories', 'question_A', 'question_B', 'answers'])
test_data = test_data[['categories', 'answers']]
nRow, nCol = test_data.shape
print(f'There are {nRow} rows and {nCol} columns')
test_data.head(10)

There are 1400000 rows and 2 columns


,categories,answers
0,5,Optical mice use an LED and a camera to rapidl...
1,6,i hear that the mojave road is amazing!<br />\...
2,3,Trans fats occur in manufactured foods during ...
3,7,according to the www.fedex.com web site:\nAir ...
4,7,renting vs buying depends on your goals. <br /...
5,5,"There are commercial kits available, but a can..."
6,2,from ask yahoo...\nhttp://ask.yahoo.com/ask/20...
7,8,Well. Everyone has different definition on wh...
8,5,"Not sure if this is the origin, but I think it..."
9,2,A tough question as it overlaps science and th...


- Eliminar las filas vacías

In [4]:
test_data.isnull().sum()

categories        0
answers       24579
dtype: int64

In [5]:
test_data = test_data.dropna().reset_index(drop=True)
test_data.isnull().sum()

categories    0
answers       0
dtype: int64

In [15]:
test_data

,categories,answers
0,5,Optical mice use an LED and a camera to rapidl...
1,6,i hear that the mojave road is amazing!<br />\...
2,3,Trans fats occur in manufactured foods during ...
3,7,according to the www.fedex.com web site:\nAir ...
4,7,renting vs buying depends on your goals. <br /...
...,...,...
1375416,3,I increased my height 2 feet afterwards so yes...
1375417,7,"ebay, electronic boutique, babbages or flea ma..."
1375418,3,It can be normal as long as they are not cance...
1375419,1,Main hindi bol sakti hoon.kahiye.


## Tokenización con NLTK

In [6]:
import nltk
import re
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
df_tokenizado = pd.DataFrame()

df_tokenizado = test_data["answers"].apply(nltk.word_tokenize)
df_tokenizado

0          [Optical, mice, use, an, LED, and, a, camera, ...
1          [i, hear, that, the, mojave, road, is, amazing...
2          [Trans, fats, occur, in, manufactured, foods, ...
3          [according, to, the, www.fedex.com, web, site,...
4          [renting, vs, buying, depends, on, your, goals...
                                 ...                        
1375416    [I, increased, my, height, 2, feet, afterwards...
1375417    [ebay, ,, electronic, boutique, ,, babbages, o...
1375418    [It, can, be, normal, as, long, as, they, are,...
1375419            [Main, hindi, bol, sakti, hoon.kahiye, .]
1375420    [★★, HELP, WITH, SEARCHING, ★★\n\n◙, I, used, ...
Name: answers, Length: 1375421, dtype: object

## Gensim con FastText y tokenización de NLTK (sin pre-procesado)

In [8]:
from gensim.models import FastText

INFO - 13:10:50: 'pattern' package not found; tag filters are not available for English


In [11]:
model_yahoo = FastText(df_tokenizado, size=100, window=5, min_count=5, workers=4,sg=1)

INFO - 13:13:40: collecting all words and their counts
INFO - 13:13:40: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 13:13:40: PROGRESS: at sentence #10000, processed 714131 words, keeping 56976 word types
INFO - 13:13:41: PROGRESS: at sentence #20000, processed 1441535 words, keeping 91269 word types
INFO - 13:13:41: PROGRESS: at sentence #30000, processed 2158886 words, keeping 120374 word types
INFO - 13:13:41: PROGRESS: at sentence #40000, processed 2846930 words, keeping 146552 word types
INFO - 13:13:41: PROGRESS: at sentence #50000, processed 3545315 words, keeping 170809 word types
INFO - 13:13:41: PROGRESS: at sentence #60000, processed 4248071 words, keeping 194287 word types
INFO - 13:13:41: PROGRESS: at sentence #70000, processed 4936280 words, keeping 216221 word types
INFO - 13:13:41: PROGRESS: at sentence #80000, processed 5633230 words, keeping 237644 word types
INFO - 13:13:42: PROGRESS: at sentence #90000, processed 6332531 words, keeping 2

In [18]:
model_yahoo.init_sims(replace=True)

INFO - 14:14:45: precomputing L2-norms of word weight vectors
INFO - 14:14:47: precomputing L2-norms of ngram weight vectors


- Buscar las palabras similares:

In [19]:
model_yahoo.wv.most_similar("ball")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('puck', 0.8929243087768555),
 ('bat', 0.8638759851455688),
 ('cueball', 0.8566311597824097),
 ('fastball', 0.8549059629440308),
 ('dribble', 0.8347573280334473),
 ('dribbles', 0.818213701248169),
 ('curveball', 0.8132420778274536),
 ('dribbler', 0.8028526902198792),
 ('9-ball', 0.8020328283309937),
 ('balls', 0.7991012334823608)]

- Otra forma:

In [20]:
q = model_yahoo.wv['artist']
model_yahoo.wv.similar_by_vector(q)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('artist', 0.9999999403953552),
 ('musician', 0.828183114528656),
 ('songwriter', 0.8101131916046143),
 ('singer/songwriter', 0.793769359588623),
 ('singer-songwriter', 0.7834258079528809),
 ('artiste', 0.7775751948356628),
 ('singer', 0.7610446214675903),
 ('Songwriter', 0.7586888670921326),
 ('artists', 0.7519974708557129),
 ('songwriters', 0.7478518486022949)]

In [21]:
model_yahoo.wv.most_similar("rock_music")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[("'music", 0.8159449100494385),
 ('music/movies', 0.8032846450805664),
 ('musics', 0.7943612337112427),
 ('music-', 0.7915775775909424),
 ('music', 0.7898300886154175),
 ('\\nmusic', 0.7672858834266663),
 ('music..', 0.7641711235046387),
 ('rock/metal', 0.7593479156494141),
 ('rock-n-roll', 0.7560387849807739),
 ('rock', 0.7423827648162842)]

In [22]:
model_yahoo.wv.most_similar("music")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('musics', 0.8669745326042175),
 ('music-', 0.8439751863479614),
 ('music..', 0.8365069031715393),
 ('music/movies', 0.8258087038993835),
 ("'music", 0.807208776473999),
 ('musicals', 0.8044894933700562),
 ('songs', 0.7936811447143555),
 ('tunes', 0.7903274297714233),
 ('musical', 0.7739101648330688),
 ('music.\\n', 0.7627118825912476)]

In [23]:
model_yahoo.wv.most_similar("rock")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[("rock'n", 0.8456147909164429),
 ('rock-', 0.8409271240234375),
 ('rock/metal', 0.8071688413619995),
 ('rocks', 0.8067232370376587),
 ('frock', 0.7981610298156738),
 ('rock-n-roll', 0.788267970085144),
 ('punk', 0.7659653425216675),
 ('folk-rock', 0.7527793645858765),
 ('rockabilly', 0.7488818764686584),
 ('raggae', 0.7406305074691772)]

## Preprocesado con Spacy

In [10]:
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing
import re

- **Limpiar las palabras**: We are lemmatizing and removing the stopwords and non-alphabetic characters for each line of dialogue.

In [7]:
nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [16]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in test_data.answers.values)

In [17]:
brief_cleaning

<generator object <genexpr> at 0x7fdbdd627200>

In [18]:
t = time()
#Usamos Pipe de Spacy para agilizar el proceso
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 43.81 mins


In [19]:
df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(1233550, 1)

In [20]:
df_clean.head()

,clean
0,optical mouse use led camera rapidly capture i...
1,hear mojave road amazing br nsearch online
2,trans fat occur manufacture food process parti...
3,accord www fedex com web site nair fleet br n ...
4,rent vs buying depend goal br ngenerally think...


We are using Gensim **Phrases** package to automatically detect common phrases (**bigrams**) from a list of sentences (mr_roger)

In [21]:
from gensim.models.phrases import Phrases, Phraser

In [22]:
sent = [row.split() for row in df_clean['clean']]

In [25]:
phrases = Phrases(sent, min_count=10, progress_per=10000)

INFO - 16:26:48: collecting all words and their counts
INFO - 16:26:48: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 16:26:49: PROGRESS: at sentence #10000, processed 335192 words and 281529 word types
INFO - 16:26:49: PROGRESS: at sentence #20000, processed 677390 words and 511955 word types
INFO - 16:26:50: PROGRESS: at sentence #30000, processed 1013219 words and 716307 word types
INFO - 16:26:50: PROGRESS: at sentence #40000, processed 1338903 words and 903882 word types
INFO - 16:26:51: PROGRESS: at sentence #50000, processed 1654611 words and 1076355 word types
INFO - 16:26:51: PROGRESS: at sentence #60000, processed 1973467 words and 1243445 word types
INFO - 16:26:52: PROGRESS: at sentence #70000, processed 2295683 words and 1406655 word types
INFO - 16:26:52: PROGRESS: at sentence #80000, processed 2618626 words and 1564001 word types
INFO - 16:26:53: PROGRESS: at sentence #90000, processed 2929069 words and 1711264 word types
INFO - 16:26:53: PROGRESS: 

- The goal of **Phraser()** is to cut down memory consumption of Phrases(), by discarding model state not strictly needed for the bigram detection task:

In [27]:
bigram = Phraser(phrases)

INFO - 16:29:17: source_vocab length 12449320
INFO - 16:29:41: Phraser added 50000 phrasegrams
INFO - 16:30:58: Phraser built with 59144 phrasegrams


In [28]:
sentences = bigram[sent]

In [65]:
sentences

- **sanity check** of the effectiveness of the lemmatization, removal of stopwords, and addition of bigrams.

In [29]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

636237

In [30]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['n',
 'like',
 'know',
 'good',
 'think',
 'time',
 'people',
 'want',
 'need',
 'thing']

## Gensim con Spacy y **Word2Vec** implementation (**Collocation detection**)

In [31]:
import multiprocessing

from gensim.models import Word2Vec

In [33]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

2

In [35]:
w2v_model = Word2Vec(min_count=10, #min_count = int - Ignores all words with total absolute frequency lower than this - (2, 100)
                     window=5, #window = int - The maximum distance between the current and predicted word within a sentence. E.g. window words on the left and window words on the left of our target - (2, 10)
                     size=100, #size = int - Dimensionality of the feature vectors. - (50, 300)
                     sample=6e-5, #sample = float - The threshold for configuring which higher-frequency words are randomly downsampled. Highly influencial. - (0, 1e-5)
                     alpha=0.03, #alpha = float - The initial learning rate - (0.01, 0.05)
                     min_alpha=0.0007, #min_alpha = float - Learning rate will linearly drop to min_alpha as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00
                     negative=20, #negative = int - If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
                     workers=cores-1)

- **Vocabulary** build

In [38]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 16:38:36: collecting all words and their counts
INFO - 16:38:36: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 16:38:37: PROGRESS: at sentence #10000, processed 306640 words, keeping 42856 word types
INFO - 16:38:38: PROGRESS: at sentence #20000, processed 619046 words, keeping 66027 word types
INFO - 16:38:39: PROGRESS: at sentence #30000, processed 925997 words, keeping 83356 word types
INFO - 16:38:40: PROGRESS: at sentence #40000, processed 1223937 words, keeping 98865 word types
INFO - 16:38:40: PROGRESS: at sentence #50000, processed 1513193 words, keeping 111762 word types
INFO - 16:38:41: PROGRESS: at sentence #60000, processed 1805548 words, keeping 123090 word types
INFO - 16:38:42: PROGRESS: at sentence #70000, processed 2101325 words, keeping 134135 word types
INFO - 16:38:43: PROGRESS: at sentence #80000, processed 2397395 words, keeping 144182 word types
INFO - 16:38:43: PROGRESS: at sentence #90000, processed 2681693 words, keeping 15351

Time to build vocab: 1.95 mins


- **Train**

  - total_examples = int - Count of sentences;
  - epochs = int - Number of iterations (epochs) over the corpus - [10, 20, 30]




In [39]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=3, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2))

INFO - 16:41:06: training model with 1 workers on 124568 vocabulary and 100 features, using sg=0 hs=0 sample=6e-05 negative=20 window=5
INFO - 16:41:07: EPOCH 1 - PROGRESS: at 0.47% examples, 112849 words/s, in_qsize 1, out_qsize 0
INFO - 16:41:08: EPOCH 1 - PROGRESS: at 0.91% examples, 107535 words/s, in_qsize 1, out_qsize 0
INFO - 16:41:09: EPOCH 1 - PROGRESS: at 1.40% examples, 111156 words/s, in_qsize 1, out_qsize 0
INFO - 16:41:10: EPOCH 1 - PROGRESS: at 1.91% examples, 111709 words/s, in_qsize 1, out_qsize 0
INFO - 16:41:11: EPOCH 1 - PROGRESS: at 2.43% examples, 113136 words/s, in_qsize 1, out_qsize 0
INFO - 16:41:12: EPOCH 1 - PROGRESS: at 2.96% examples, 113805 words/s, in_qsize 1, out_qsize 0
INFO - 16:41:13: EPOCH 1 - PROGRESS: at 3.48% examples, 114054 words/s, in_qsize 1, out_qsize 0
INFO - 16:41:14: EPOCH 1 - PROGRESS: at 4.02% examples, 114648 words/s, in_qsize 2, out_qsize 0
INFO - 16:41:15: EPOCH 1 - PROGRESS: at 4.55% examples, 114731 words/s, in_qsize 1, out_qsize 0


Time to train the model: 9.92 mins


- Save model

In [40]:
w2v_model.save("/content/drive/My Drive/Colab Notebooks/Data/Yahoo_Answers/w2v_model.model")

INFO - 17:00:16: saving Word2Vec object under /content/drive/My Drive/Colab Notebooks/Data/Yahoo_Answers/w2v_model.model, separately None
INFO - 17:00:16: storing np array 'vectors' to /content/drive/My Drive/Colab Notebooks/Data/Yahoo_Answers/w2v_model.model.wv.vectors.npy
INFO - 17:00:16: not storing attribute vectors_norm
INFO - 17:00:16: storing np array 'syn1neg' to /content/drive/My Drive/Colab Notebooks/Data/Yahoo_Answers/w2v_model.model.trainables.syn1neg.npy
INFO - 17:00:17: not storing attribute cum_table
INFO - 17:00:17: saved /content/drive/My Drive/Colab Notebooks/Data/Yahoo_Answers/w2v_model.model


- Optimize use if not going to train again

In [41]:
w2v_model.init_sims(replace=True)

INFO - 17:00:45: precomputing L2-norms of word weight vectors


- Search for words

In [42]:
w2v_model.wv.most_similar(positive=["ball"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ball_hit', 0.8803877234458923),
 ('hit_ball', 0.865219235420227),
 ('throw_ball', 0.8581801652908325),
 ('ball_bounce', 0.8462117910385132),
 ('tennis_ball', 0.8153362274169922),
 ('dribble', 0.8057109117507935),
 ('kick_ball', 0.7863442301750183),
 ('cue_ball', 0.7803047895431519),
 ('volley', 0.7795623540878296),
 ('scrum', 0.768436074256897)]

In [43]:
w2v_model.wv.most_similar(positive=["artist"])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('musician', 0.8039395809173584),
 ('record_label', 0.7966662049293518),
 ('songwriter', 0.77672278881073),
 ('music_industry', 0.7668749094009399),
 ('indie', 0.7574900984764099),
 ('singer', 0.7381908893585205),
 ('song_album', 0.7219146490097046),
 ('album', 0.7116407155990601),
 ('beatle', 0.7079148292541504),
 ('music', 0.7075743079185486)]

In [45]:
w2v_model.wv.most_similar("rock_music")

KeyError: ignored

In [47]:
w2v_model.wv.most_similar("rock")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('nrock', 0.6751759052276611),
 ('pebble', 0.6520676612854004),
 ('metamorphic_rock', 0.649256706237793),
 ('igneous_rock', 0.6480199694633484),
 ('sedimentary_rock', 0.635589599609375),
 ('rock_roll', 0.633292555809021),
 ('granite', 0.6182916760444641),
 ('sandstone', 0.6163721680641174),
 ('grunge', 0.6150509119033813),
 ('rock_band', 0.6127825379371643)]

In [48]:
w2v_model.wv.most_similar("music")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('classical_music', 0.8612747192382812),
 ('genre_music', 0.7764829397201538),
 ('hip_hop', 0.7708561420440674),
 ('techno', 0.7631326913833618),
 ('play_instrument', 0.7590484023094177),
 ('musical', 0.7482472658157349),
 ('genre', 0.7467389106750488),
 ('nmusic', 0.7451083064079285),
 ('song', 0.7425194978713989),
 ('reggae', 0.7407678365707397)]

In [54]:
w2v_model.wv.most_similar("God")

KeyError: ignored

In [55]:
w2v_model.wv.most_similar("god")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ngod', 0.8592567443847656),
 ('bible_say', 0.8580961227416992),
 ('almighty', 0.8483306169509888),
 ('worship_god', 0.8474438190460205),
 ('god_almighty', 0.8390748500823975),
 ('eternally', 0.8387427926063538),
 ('godly', 0.8326869010925293),
 ('son_god', 0.8314530253410339),
 ('jehovah_god', 0.8282667398452759),
 ('almighty_god', 0.8267010450363159)]

## Gensim con Spacy y FastText

In [66]:
import multiprocessing

from gensim.models import FastText

In [67]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

2

In [71]:
ft_spacy_model = FastText(min_count=10, #min_count = int - Ignores all words with total absolute frequency lower than this - (2, 100)
                     window=5, #window = int - The maximum distance between the current and predicted word within a sentence. E.g. window words on the left and window words on the left of our target - (2, 10)
                     size=100, #size = int - Dimensionality of the feature vectors. - (50, 300)
                     sample=6e-5, #sample = float - The threshold for configuring which higher-frequency words are randomly downsampled. Highly influencial. - (0, 1e-5)
                     alpha=0.03, #alpha = float - The initial learning rate - (0.01, 0.05)
                     min_alpha=0.0007, #min_alpha = float - Learning rate will linearly drop to min_alpha as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00
                     negative=20, #negative = int - If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
                     workers=cores-1)

- Vocab build

In [72]:
t = time()

ft_spacy_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 17:40:20: collecting all words and their counts
INFO - 17:40:20: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 17:40:21: PROGRESS: at sentence #10000, processed 306640 words, keeping 42856 word types
INFO - 17:40:21: PROGRESS: at sentence #20000, processed 619046 words, keeping 66027 word types
INFO - 17:40:22: PROGRESS: at sentence #30000, processed 925997 words, keeping 83356 word types
INFO - 17:40:23: PROGRESS: at sentence #40000, processed 1223937 words, keeping 98865 word types
INFO - 17:40:24: PROGRESS: at sentence #50000, processed 1513193 words, keeping 111762 word types
INFO - 17:40:24: PROGRESS: at sentence #60000, processed 1805548 words, keeping 123090 word types
INFO - 17:40:25: PROGRESS: at sentence #70000, processed 2101325 words, keeping 134135 word types
INFO - 17:40:26: PROGRESS: at sentence #80000, processed 2397395 words, keeping 144182 word types
INFO - 17:40:27: PROGRESS: at sentence #90000, processed 2681693 words, keeping 15351

Time to build vocab: 2.07 mins


- Train

In [78]:
t = time()

ft_spacy_model.train(sentences=sentences, total_examples=len(sentences), epochs=3, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 17:50:08: training model with 1 workers on 124568 vocabulary and 100 features, using sg=0 hs=0 sample=6e-05 negative=20 window=5
INFO - 17:50:09: EPOCH 1 - PROGRESS: at 0.26% examples, 62243 words/s, in_qsize 1, out_qsize 0
INFO - 17:50:10: EPOCH 1 - PROGRESS: at 0.55% examples, 63882 words/s, in_qsize 1, out_qsize 0
INFO - 17:50:11: EPOCH 1 - PROGRESS: at 0.84% examples, 64239 words/s, in_qsize 2, out_qsize 0
INFO - 17:50:12: EPOCH 1 - PROGRESS: at 1.12% examples, 64738 words/s, in_qsize 1, out_qsize 0
INFO - 17:50:13: EPOCH 1 - PROGRESS: at 1.40% examples, 64794 words/s, in_qsize 1, out_qsize 0
INFO - 17:50:14: EPOCH 1 - PROGRESS: at 1.70% examples, 64790 words/s, in_qsize 1, out_qsize 0
INFO - 17:50:15: EPOCH 1 - PROGRESS: at 1.99% examples, 64885 words/s, in_qsize 1, out_qsize 0
INFO - 17:50:16: EPOCH 1 - PROGRESS: at 2.27% examples, 64959 words/s, in_qsize 1, out_qsize 0
INFO - 17:50:17: EPOCH 1 - PROGRESS: at 2.57% examples, 64945 words/s, in_qsize 1, out_qsize 0
INFO - 17

Time to train the model: 17.24 mins


In [80]:
ft_spacy_model.wv.most_similar("God")

INFO - 18:08:01: precomputing L2-norms of word weight vectors
INFO - 18:08:01: precomputing L2-norms of ngram weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ye', 0.6468883752822876),
 ('lamb_god', 0.62020343542099),
 ('thy_god', 0.6175299286842346),
 ('ggod', 0.604841947555542),
 ('bless_sabbath', 0.6041961312294006),
 ('say_unto', 0.6033000946044922),
 ('woah', 0.6027402877807617),
 ('lust_flesh', 0.5909445285797119),
 ('vah', 0.5905077457427979),
 ('god_merciful', 0.587104082107544)]

In [81]:
ft_spacy_model.wv.most_similar("god")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('ggod', 0.9615902900695801),
 ('godse', 0.9292232990264893),
 ('obey_god', 0.8943073153495789),
 ('almighty_god', 0.8830417394638062),
 ('jehovah_god', 0.881996750831604),
 ('wrath_god', 0.8762962222099304),
 ('son_god', 0.8760213851928711),
 ('god_hath', 0.8726826906204224),
 ('thy_god', 0.8664657473564148),
 ('glorify_god', 0.865591287612915)]

In [82]:
ft_spacy_model.wv.most_similar("ball")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cue_ball', 0.9535918235778809),
 ('ballz', 0.9493148326873779),
 ('foul_ball', 0.9461005926132202),
 ('grip_ball', 0.9331732988357544),
 ('vball', 0.9252342581748962),
 ('toss_ball', 0.9246894121170044),
 ('bat_ball', 0.9232808351516724),
 ('throw_ball', 0.9219976663589478),
 ('bowl_ball', 0.921135425567627),
 ('hit_ball', 0.919243574142456)]

In [83]:
ft_spacy_model.wv.most_similar("Ball")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[("ny'all", 0.7827248573303223),
 ('squall', 0.7429753541946411),
 ('fumble_ball', 0.7399603128433228),
 ('hardball', 0.7394171953201294),
 ('vball', 0.7390828132629395),
 ('wiffle_ball', 0.7298623919487),
 ('climb_wall', 0.729516863822937),
 ('lowball', 0.7269291281700134),
 ('rexall', 0.7260594367980957),
 ('tee_ball', 0.7234928607940674)]

- Save model

In [84]:
ft_spacy_model.save("/content/drive/My Drive/Colab Notebooks/Data/Yahoo_Answers/ft_spacy_model.model")

INFO - 18:10:14: saving FastText object under /content/drive/My Drive/Colab Notebooks/Data/Yahoo_Answers/ft_spacy_model.model, separately None
INFO - 18:10:14: storing np array 'vectors' to /content/drive/My Drive/Colab Notebooks/Data/Yahoo_Answers/ft_spacy_model.model.wv.vectors.npy
INFO - 18:10:14: not storing attribute vectors_norm
INFO - 18:10:14: storing np array 'vectors_vocab' to /content/drive/My Drive/Colab Notebooks/Data/Yahoo_Answers/ft_spacy_model.model.wv.vectors_vocab.npy
INFO - 18:10:14: storing np array 'vectors_ngrams' to /content/drive/My Drive/Colab Notebooks/Data/Yahoo_Answers/ft_spacy_model.model.wv.vectors_ngrams.npy
INFO - 18:10:15: not storing attribute vectors_ngrams_norm
INFO - 18:10:15: not storing attribute vectors_vocab_norm
INFO - 18:10:15: not storing attribute buckets_word
INFO - 18:10:15: storing np array 'syn1neg' to /content/drive/My Drive/Colab Notebooks/Data/Yahoo_Answers/ft_spacy_model.model.trainables.syn1neg.npy
INFO - 18:10:15: storing np array 